In [1]:
import pandas as pd
import os

In [2]:
base_dir = '../gid_cid_map/'
all_gid_cid = pd.DataFrame()
with os.scandir(base_dir) as prime_dir:
    for entry in prime_dir:
        x = pd.read_csv(entry)
        all_gid_cid = all_gid_cid.append(x, ignore_index = True)
all_gid_cid['GID'] = 'GID' + all_gid_cid['GID'].astype('str')

In [3]:
all_gid = set(all_gid_cid['GID'].tolist())
print(len(all_gid))
all_gid_cid['CID'] = all_gid_cid['CID'].astype('str')
all_gid_cid['SID'] = all_gid_cid['SID'].astype('str')
all_gid_cid.head()

5020


,CID,SID,GID
0,61665,1,GID304660
1,525424,84,GID6341870
2,557028,13,GID6933502
3,568630,33,GID7400769
4,583298,79,GID8048669


In [4]:
# genotyped = pd .read_csv('beagle_miss70_cimmyt2013_2020-SNPs-1filter-passed.hmp.txt', delimiter='\t', header= None)

In [5]:
with open('../genotypic_data/beagle_miss70_cimmyt2013_2020-SNPs-1filter-passed.hmp.txt', 'r') as file:
    
    x = []
#     for line in file:
#         x.append(line.decode().split('\t'))
    #file.readline()
    genotyped = file.readline()
    genotyped = genotyped.strip().split('\t')
    

In [6]:
common_geno = all_gid.intersection(set(genotyped))
common_geno = list(common_geno)
common_geno.append('rs#')
common_geno.append('alleles')
print(len(common_geno))

3117


In [7]:
with open('../genotypic_data/beagle_miss70_cimmyt2013_2020-SNPs-1filter-passed.hmp.txt', 'r') as file:
    
      data = pd.read_csv(file, delimiter='\t', usecols = common_geno)  

In [8]:
print(data.shape)

(40767, 3117)


In [9]:
data.to_csv('../genotypic_data/extracted_data.csv', sep='\t')

In [10]:
data.head()

,rs#,alleles,GID6680533,GID6681871,GID6415882,GID6681889,GID7401114,GID6414558,GID6415858,GID7629157,...,GID8045774,GID7310290,GID6681899,GID8054193,GID8054192,GID8054189,GID6680599,GID6331702,GID6681851,GID6681853
0,1A_1145442,A/G,G,A,A,A,R,A,A,A,...,A,A,A,A,A,A,A,A,G,R
1,1A_1158042,T/C,C,T,T,T,C,T,T,T,...,T,T,T,T,T,T,C,T,C,Y
2,1A_1158055,G/T,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,T,G,G,G
3,1A_1230946,A/C,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,C,A,A,A
4,1A_1232964,G/A,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G


In [11]:
data_transpose = data.set_index('rs#').T

In [12]:
#data_transpose.reset_index(inplace=True)

In [13]:
data_transpose.head()

rs#,1A_1145442,1A_1158042,1A_1158055,1A_1230946,1A_1232964,1A_1236254,1A_1238074,1A_1238114,1A_1588248,1A_1698908,...,UN_473655057,UN_473655073,UN_474138840,UN_474138845,UN_474138866,UN_474767314,UN_475092207,UN_475092295,UN_476929267,UN_476929292
alleles,A/G,T/C,G/T,A/C,G/A,A/C,A/G,G/A,G/A,G/T,...,G/C,G/A,C/T,C/T,G/A,A/G,A/C,A/G,A/C,T/C
GID6680533,G,C,G,A,G,A,A,G,G,G,...,G,G,C,C,G,A,A,A,A,T
GID6681871,A,T,G,A,G,A,A,G,G,G,...,G,G,C,C,G,G,A,A,A,T
GID6415882,A,T,G,A,G,A,A,G,G,G,...,G,G,C,C,G,G,M,A,A,T
GID6681889,A,T,G,A,G,A,A,G,G,G,...,G,G,C,C,G,R,A,A,A,T


In [14]:
#data_transpose = data_transpose.merge(all_gid_cid, how='left', left_on='index', right_on='GID')

In [15]:
#data_transpose.head()

In [16]:
data_transpose.to_pickle('../genotypic_data/sequence_data.pkl')

In [62]:
cols = data_transpose.columns

In [63]:
for col in cols:
    allele = data_transpose.loc['alleles', col]
    allele = allele.split('/')
    data_transpose[col][(data_transpose[col] != allele[0]) & (data_transpose[col] != allele[1])] = 0
    data_transpose[col][data_transpose[col] == allele[0]] = 1
    data_transpose[col][data_transpose[col] == allele[1]] = -1
    

In [64]:
data_transpose.reset_index(inplace=True)

data_transpose.iloc[1:,:].head()


rs#,index,1A_1145442,1A_1158042,1A_1158055,1A_1230946,1A_1232964,1A_1236254,1A_1238074,1A_1238114,1A_1588248,...,UN_473655057,UN_473655073,UN_474138840,UN_474138845,UN_474138866,UN_474767314,UN_475092207,UN_475092295,UN_476929267,UN_476929292
1,GID6680533,-1,-1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,GID6681871,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,-1,1,1,1,1
3,GID6415882,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,-1,0,1,1,1
4,GID6681889,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
5,GID7401114,0,-1,1,1,1,1,1,1,1,...,1,1,0,0,1,-1,1,1,1,1


In [65]:
data_transpose = data_transpose.rename(columns={'index': 'GID'})
data_transpose.iloc[1:,:].head()

rs#,GID,1A_1145442,1A_1158042,1A_1158055,1A_1230946,1A_1232964,1A_1236254,1A_1238074,1A_1238114,1A_1588248,...,UN_473655057,UN_473655073,UN_474138840,UN_474138845,UN_474138866,UN_474767314,UN_475092207,UN_475092295,UN_476929267,UN_476929292
1,GID6680533,-1,-1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,GID6681871,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,-1,1,1,1,1
3,GID6415882,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,-1,0,1,1,1
4,GID6681889,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
5,GID7401114,0,-1,1,1,1,1,1,1,1,...,1,1,0,0,1,-1,1,1,1,1


In [66]:
data_transpose.iloc[1:,:].to_csv('../genotypic_data/encoded_data.csv', sep='\t', index=False)